In [19]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

load_dotenv()
KEY = os.getenv('API_KEY')

if not KEY:
    raise ValueError("API ключ не найден. Проверьте файл .env и убедитесь, что переменная API_KEY задана.")

url_history = "https://api.apilayer.com/currency_data/historical"

headers = {
    "apikey": KEY
}

start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 1, 7)

In [21]:
data_list = []
date_list = []
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date.strftime('%Y-%m-%d'))
    current_date += timedelta(days=1)

# Перебор дат и сбор данных
for date_str in date_list:
    # Параметры запроса
    params = {
        'date': date_str
    }
    
    response = requests.get(url_history, headers=headers, params=params)
    
    # get 200
    if response.status_code == 200:
        data = response.json()
        
        # Проверяем наличие ключа 'quotes' в ответе
        if 'quotes' in data:
            quotes = data['quotes']
            
            # Если это первый запрос, сохраняем первые 10 валют
            if len(data_list) == 0:
                currencies = list(quotes.keys())[:10]
            
            rate_dict = {'date': date_str}
            for currency in currencies:
                rate_dict[currency] = quotes.get(currency, None)
            
            data_list.append(rate_dict)
        else:
            print(f"В ответе на дату {date_str} отсутствует ключ 'quotes'.")
    else:
        print(f"Ошибка при получении данных на дату {date_str}: {response.status_code}")
        print(f"Тело ответа: {response.text}")



In [22]:
currency_df = pd.DataFrame(data_list)

# Устанавливаем дату как индекс
currency_df.set_index('date', inplace=True)
print(currency_df)

              USDAED      USDAFN      USDALL      USDAMD    USDANG  \
date                                                                 
2022-01-01  3.673042  103.750404  106.650403  480.234325  1.802591   
2022-01-02  3.672980  103.749983  106.650326  480.234325  1.802591   
2022-01-03  3.673103  103.750299  106.249880  481.590005  1.802973   
2022-01-04  3.673096  104.899369  105.874950  482.301630  1.801500   
2022-01-05  3.673102  105.250158  106.902530  483.164239  1.802797   
2022-01-06  3.673102  104.999962  107.350118  483.038449  1.802366   
2022-01-07  3.673042  105.000368  107.000368  483.788207  1.802362   

                USDAOA      USDARS    USDAUD  USDAWG    USDAZN  
date                                                            
2022-01-01  550.590403  102.689974  1.375800  1.8000  1.703970  
2022-01-02  550.590435  102.664691  1.376179  1.8000  1.698647  
2022-01-03  554.203977  102.958502  1.389502  1.8005  1.701522  
2022-01-04  553.427992  103.104100  1.381105